## Import the basic libraries

In [ ]:
import time
from qiskit import QuantumCircuit
import qiskit.qasm3
from qbraid import QbraidProvider
from qbraid.exceptions import QbraidError

## Install qbraid

In [ ]:
! pip install qbraid
! pip install qbraid-cli

## Setting up your account

In [ ]:
! qbraid configure --api-key <your_api_key>

## A minimal example

First, we set up the backend. For this, we first print the devices that are available and then select one from the list.

In [ ]:
from qbraid import QbraidProvider

def list_backends():
    # 1. Connect to qBraid
    provider = QbraidProvider()

    # 2. Get all devices
    # You can pass filters like: provider.get_devices(filters={"status": "ONLINE"})
    devices = provider.get_devices()

    print(f"{'DEVICE ID':<35} {'PROVIDER':<15} {'STATUS':<10}")
    print("-" * 65)

    for device in devices:
        # Access device metadata
        # Note: In some versions, use device.metadata()['device_id']
        # but usually device.id works directly.
        dev_id = device.id
        status = device.status()
        
        # Determine provider name (usually part of the object or metadata)
        # We can often parse it from the ID or class name
        provider_name = dev_id.split(':')[0] if ':' in dev_id else "qBraid/Other"

        print(f"{dev_id:<35} {provider_name:<15} {status:<10}")

if __name__ == "__main__":
    list_backends()

## Transpile the circuit

We can now create a circuit and transpile it for the selected backend. Transpilation is the process of converting a quantum circuit into a form that can be executed on a specific quantum device. This involves optimizing the circuit and mapping it to the available qubits and gates of the device.

In [ ]:
from qbraid.transpiler import transpile

def run_bell_state():
    # --- STEP 1: Define the Algorithm (Bell State) ---
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    
    print("Circuit Created:")
    print(qc)

    # --- STEP 2: Connect to qBraid ---
    try:
        provider = QbraidProvider()
        
        # Using qBraid QIR Simulator
        device_id = "qbraid:qbraid:sim:qir-sv" 
        device = provider.get_device(device_id)
        
        print(f"\n--- Sending to qBraid Device: {device_id} ---")

    except QbraidError as e:
        print(f"\nError connecting to qBraid: {e}")
        return

    # --- STEP 3: Run the Job ---
    # FIX: Manually convert Qiskit circuit to OpenQASM 3 string
    # This bypasses the missing automatic conversion path in the SDK.
    try:
        qasm3_program = qiskit.qasm3.dumps(qc)
        job = device.run(qasm3_program, shots=1024)
    except Exception as e:
        print(f"Submission failed: {e}")
        return
    
    print(f"Job Submitted! ID: {job.id}")
    
    # Wait for results
    print("Waiting for results...", end="", flush=True)
    while job.status() not in ['COMPLETED', 'FAILED', 'CANCELLED']:
        time.sleep(1)
        print(".", end="", flush=True)
    
    print("\n")

    # --- STEP 4: Print Results ---
    if job.status() == 'COMPLETED':
        counts = job.result().measurement_counts()
        print("Success! Measurement Counts:")
        print(counts)
    else:
        print("Job failed.")

if __name__ == "__main__":
    run_bell_state()

## Submit the job

In [ ]:
import time
from qiskit import QuantumCircuit
from qbraid import QbraidProvider
from qbraid.exceptions import QbraidError

def run_aws_dm1():
    # --- STEP 1: Define the Algorithm ---
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)

    print("Circuit Created (Qiskit):")
    print(qc)

    # --- STEP 2: Connect to qBraid ---
    try:
        provider = QbraidProvider()
        
        # TARGET: AWS Density Matrix Simulator
        device_id = "aws:aws:sim:dm1"
        device = provider.get_device(device_id)
        
        print(f"\n--- Sending to qBraid Device: {device_id} ---")

    except QbraidError as e:
        print(f"Error connecting: {e}")
        return

    # --- STEP 3: Run the Job ---
    try:
        # AWS devices via qBraid accept Qiskit objects directly.
        # The SDK automatically transpiles Qiskit -> Braket.
        job = device.run(qc, shots=1024)
        braket_circuit = transpile(qc, target="braket")
        job = device.run(braket_circuit, shots=1024)
        
        print(f"Job Submitted! ID: {job.id}")
        
    except Exception as e:
        print(f"\nSubmission failed. You might need to install the AWS plugin:")
        print(f"Try running: pip install 'qbraid[aws]'")
        print(f"Error details: {e}")
        return
    
    # --- STEP 4: Wait for Results ---
    # AWS queues can be slower than local simulators.
    print("Waiting for results (AWS queue)...", end="", flush=True)
    
    status = job.status()
    while status not in ['COMPLETED', 'FAILED', 'CANCELLED']:
        time.sleep(2) # Poll less frequently for cloud jobs
        status = job.status()
        print(".", end="", flush=True)
    
    print("\n")

    # --- STEP 5: Print Results ---
    if status == 'COMPLETED':
        counts = job.result().measurement_counts()
        print("Success! Measurement Counts:")
        print(counts)
    else:
        print("Job failed.")

if __name__ == "__main__":
    run_aws_dm1()